### Passo 1 - Importar Arquivos e Bibliotecas

In [19]:
import pandas as pd
from pathlib import Path
import win32com.client as win32

In [20]:
email = pd.read_excel('Bases de Dados/Emails.xlsx')
lojas = pd.read_csv('Bases de Dados/Lojas.csv', encoding = "ISO-8859-1", sep=';')
vendas = pd.read_excel('Bases de Dados/Vendas.xlsx')
list_colu = ['Código Venda', 'Data', 'ID Loja', 'Loja', 'Produto', 'Quantidade', 'Valor Unitário', 'Valor Final']
vendas = pd.merge(vendas, lojas, on= 'ID Loja')
vendas=vendas.reindex(columns=list_colu)

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [21]:
dicionarios_loja = {}
for loja in lojas['Loja']:
    dicionarios_loja[loja] = vendas.loc[vendas['Loja']==loja, :]


In [22]:
dia_indicador = vendas['Data'].max()


### Passo 3 - Salvar a planilha na pasta de backup

In [30]:
# CRIANDO PASTAS DAS LOJAS
caminho = Path(r'Backup Arquivos Lojas')
if caminho.exists() == False:
    caminho.mkdir()
#MEU CODIGO
for loja in dicionarios_loja:
    arquivo = caminho / loja
    if arquivo.exists() == False:
        arquivo.mkdir()
   
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.day, dia_indicador.month, loja)
    local_arquivo = caminho / loja / nome_arquivo
    dicionarios_loja[loja].to_excel(local_arquivo)


### Passo 4 - Adicionando um email para cada gerente na tabela "Email"

In [24]:
#Criando um email para cada gerente

contador = 0
for gmail in email['E-mail']:
    email['E-mail'][contador] = 'schwartzcesarevictorgames+{}@gmail.com'.format(email['Gerente'][contador])
    contador += 1
email

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,schwartzcesarevictorgames+Helena@gmail.com
1,Shopping Midway Mall,Alice,schwartzcesarevictorgames+Alice@gmail.com
2,Norte Shopping,Laura,schwartzcesarevictorgames+Laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,schwartzcesarevictorgames+Manuela@gmail.com
4,Shopping União de Osasco,Valentina,schwartzcesarevictorgames+Valentina@gmail.com
5,Shopping Center Interlagos,Sophia,schwartzcesarevictorgames+Sophia@gmail.com
6,Rio Mar Recife,Isabella,schwartzcesarevictorgames+Isabella@gmail.com
7,Salvador Shopping,Heloisa,schwartzcesarevictorgames+Heloisa@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,schwartzcesarevictorgames+Luiza@gmail.com
9,Shopping Center Leste Aricanduva,Julia,schwartzcesarevictorgames+Julia@gmail.com


### Passo 5 - Calculando os Indicadores e enviandos os emails

In [25]:
#Metas

faturamento_meta_dia = 1000
faturamento_meta_ano = 1650000
diversidade_meta_dia = 4
diversidade_meta_ano = 120
ticket_meta_dia = 500
ticket_meta_ano = 500

#CALCULAR FATURAMENTO

for loja in dicionarios_loja:
    venda_loja = dicionarios_loja[loja]
    vendas_loja_dia = venda_loja.loc[venda_loja['Data'] == dia_indicador, :]

    faturamento_ano = venda_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #CALCULAR DIVERSIDADES DE PRODUTOS
    diversidade_ano = len(venda_loja['Produto'].unique())
    diversidade_dia = len(vendas_loja_dia['Produto'].unique())

    #TICKET MEDIO
    ticket_ano = venda_loja.groupby('Código Venda').sum()
    ticket_medio_ano = ticket_ano['Valor Final'].mean()

    ticket_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = ticket_dia['Valor Final'].mean()
    
        #cors
    if faturamento_dia >= faturamento_meta_dia:
        fat_color_dia = 'green'
    else:
        fat_color_dia = 'red'

    if faturamento_ano >= faturamento_meta_ano:
        fat_color_ano = 'green'
    else:
        fat_color_ano = 'red'

    if diversidade_dia >= diversidade_meta_dia:
        dive_color_dia = 'green'
    else:
        dive_color_dia = 'red'

    if diversidade_ano >= diversidade_meta_ano:
        dive_color_ano = 'green'
    else:
        dive_color_ano = 'red'

    if ticket_medio_dia >= ticket_meta_dia:
        tick_color_dia = 'green'
    else:
        tick_color_dia = 'red'

    if ticket_medio_ano >= ticket_meta_ano:
        tick_color_ano = 'green'
    else:
        tick_color_ano = 'red'
        
    #Enviando um email para cada gerente 
    
    gerente = email.loc[email['Loja']==loja,'Gerente'].values[0]
    gmail = email.loc[email['Loja']==loja,'E-mail'].values[0]
    dia = 'DATA: ({}/{}/{})'.format(dia_indicador.day, dia_indicador.month, dia_indicador.year)
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = gmail
    mail.Subject = 'OnePage referente a {}'.format(dia)
    mail.HTMLBody = f'''
    <p>Bom dia,<strong>{gerente}</strong></p>

    <p>O resultado de ontem <strong>(dia {dia_indicador.day}/{dia_indicador.month})</strong> da <strong>{loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cénario Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">{faturamento_dia:,.2f}</td>
        <td style="text-align: center">{faturamento_meta_dia:,.2f}</td>
        <td style="text-align: center"><font color="{fat_color_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de produtos</td>
        <td style="text-align: center">{diversidade_dia}</td>
        <td style="text-align: center">{diversidade_meta_dia}</td>
        <td style="text-align: center"><font color="{dive_color_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">{ticket_medio_dia:,.2f}</td>
        <td style="text-align: center">{ticket_meta_dia:,.2f}</td>
        <td style="text-align: center"><font color="{tick_color_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cénario Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">{faturamento_ano:,.2f}</td>
        <td style="text-align: center">{faturamento_meta_ano:,.2f}</td>
        <td style="text-align: center"><font color="{fat_color_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de produtos</td>
        <td style="text-align: center">{diversidade_ano}</td>
        <td style="text-align: center">{diversidade_meta_ano}</td>
        <td style="text-align: center"><font color="{dive_color_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">{ticket_medio_ano:,.2f}</td>
        <td style="text-align: center">{ticket_meta_ano:,.2f}</td>
        <td style="text-align: center"><font color="{tick_color_ano}">◙</font></td>
      </tr>
    </table>
    <p>att..analyst</p>
    '''

    anexos = Path.cwd() / caminho / loja / '{}_{}_{}.xlsx'.format(dia_indicador.day, dia_indicador.month, loja)
    mail.Attachments.Add(str(anexos))
    mail.Send()

### Passo 6 - Criar ranking para diretoria

In [32]:
caminho_diretoria = Path('Backup Arquivos Lojas\Diretoria')
if caminho_diretoria.exists() == False:
    caminho_diretoria.mkdir()

faturamento_ano = vendas.groupby('Loja')[['Valor Final']].sum()
faturamento_ano = faturamento_ano.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_ranking Ano.xlsx'.format(dia_indicador.day, dia_indicador.month)
faturamento_ano.to_excel(r'Backup Arquivos Lojas\Diretoria\{}'.format(nome_arquivo))

faturamento_dia_dire = vendas.loc[vendas['Data'] == dia_indicador, :]
faturamento_dia = faturamento_dia_dire.groupby('Loja')[['Valor Final']].sum()
faturamento_dia = faturamento_dia.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_ranking Dia.xlsx'.format(dia_indicador.day, dia_indicador.month)
faturamento_dia.to_excel(r'Backup Arquivos Lojas\Diretoria\{}'.format(nome_arquivo))

### Passo 7 - Enviar e-mail para diretoria

In [33]:
        
outlook=win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)

mail.To = email.loc[email['Loja'] == 'Diretoria', 'E-mail'].values[0]

mail.Subject = 'Ranking de lojas referente a DATA:{}/{}'.format(dia_indicador.day, dia_indicador.month)
mail.body = f'''
Bom dia, Diretoria

Melhor loja do dia em faturamento: Loja {faturamento_dia.index[0]} com faturamento de R${faturamento_dia.iloc[0, 0]:,.2f}
Pior loja do dia em faturamento: Loja {faturamento_dia.index[-1]} com faturamento de R${faturamento_dia.iloc[-1, 0]:,.2f}

Melhor loja do ano em faturamento: Loja {faturamento_ano.index[0]} com faturamento de R${faturamento_ano.iloc[0, 0]:,.2f}
Pior loja do ano em faturamento: Loja {faturamento_ano.index[-1]} com faturamento de R${faturamento_ano.iloc[-1, 0]:,.2f}

Prezados, segue o anexo para mais detalhes,
qualquer duvida estou a disposiçao

att...Scar
'''

anexos = Path.cwd() / caminho / 'Diretoria' / '{}_{}_ranking Ano.xlsx'.format(dia_indicador.day, dia_indicador.month)
mail.Attachments.Add(str(anexos))
anexos = Path.cwd() / caminho / 'Diretoria' / '{}_{}_ranking Dia.xlsx'.format(dia_indicador.day, dia_indicador.month)
mail.Attachments.Add(str(anexos))
mail.Send()
print('gmail enviado')

gmail enviado
